In [ ]:
from sshtunnel import SSHTunnelForwarder
import requests
import urllib.parse
import yaml
import pandas as pd
import time
import os

In [ ]:
server = SSHTunnelForwarder(
    ssh_address_or_host='rch-ds-analysis-01.int.collectivemedicaltech.com',
    ssh_username="brandon.butler",
    ssh_password="GGwp9119!",
    ssh_pkey="~/.ssh/id_rsa", 
    ssh_private_key_password="GGwp9119!",
    remote_bind_address=('127.0.0.1', 8000),
)

In [ ]:
server.start()

In [ ]:
query_engine = 'duckdb'
query_string = urllib.parse.quote('select * from facility limit 1')


r = requests.get(f'http://127.0.0.1:8000/submit_query/{query_engine}/{query_string}')
print(r.headers)
print(r.json())


In [ ]:
server.stop()

In [53]:
class DataWarehouse():
    """
    """
    __remote_host:str = 'rch-ds-analysis-01.int.collectivemedicaltech.com'
    __ssh_pkey:str = "~/.ssh/id_rsa"
    __remote_bind_address:str = '127.0.0.1'
    __remote_bind_port: int = 8000


    def __init__(self) -> None:
        with open('secrets.yaml') as secrets:
            self.__secrets = yaml.safe_load(secrets)

    def execute_query(self, query_engine: str, query_string: str) -> pd.DataFrame:
        """
        Submits a query to duckdb by:
            * establishing sshtunnel to vm running fastapi
            * passing query to fastapi
            * fastapi then submits query to duckdb
            * fast api stores query results at default locations on both vm and encrypted drive
            * query results filepaths returned here
        """

        with SSHTunnelForwarder(
            ssh_address_or_host=self.__remote_host,
            ssh_username=self.__secrets['credentials']['ssh_username'],
            ssh_password=self.__secrets['credentials']['ssh_password'],
            ssh_pkey=self.__ssh_pkey, 
            ssh_private_key_password=self.__secrets['credentials']['ssh_key_password'],
            remote_bind_address=(self.__remote_bind_address,self.__remote_bind_port)
        )as server:
            response = requests.get(f'http://127.0.0.1:8000/submit_query/{query_engine}/{query_string}').json()
        
        print(response)

        while not os.path.isfile(response['encrypted_query_results_filepath']):
            time.sleep(1)

        df = pd.read_parquet(response['encrypted_query_results_filepath'])
        return df


In [54]:
dw = DataWarehouse()

In [56]:
query_engine = "duckdb"
query_string = "SELECT * from FACILITY;"
facility = dw.execute_query(query_engine, query_string)

2022-11-03 10:15:48,520| ERROR   | Password is required for key C:\Users\butleb/.ssh\id_rsa


{'vm_query_results_filepath': '/data/temp/query_results/results_2022-11-03_11-15-49.parquet', 'encrypted_query_results_filepath': 'X:/data/temp/query_results/results_2022-11-03_11-15-49.parquet'}


In [ ]:
dir = '/data/temp/query_results'
dir = os.path.join('X:', dir)
dir

In [57]:
facility.shape

(46787, 22)